# Setup

In [2]:
from pymongo import MongoClient

In [3]:
db = MongoClient().aggregation_example

In [4]:
result = db.things.insert_many([{'x': 1, 'tags': ['dog', 'cat']},
                                {'x': 2, 'tags': ['cat']},
                                {'x': 2, 'tags': ['mouse', 'cat', 'dog']},
                                {'x': 3, 'tags': []}])

In [5]:
result.inserted_ids

[ObjectId('59a627b5498ab103579e640e'),
 ObjectId('59a627b5498ab103579e640f'),
 ObjectId('59a627b5498ab103579e6410'),
 ObjectId('59a627b5498ab103579e6411')]

# Aggregation Framework

In [6]:
from bson.son import SON

In [15]:
pipeline = [
    {'$unwind': '$tags'},
    {'$group': {'_id': '$tags', 'count': {'$sum': 1}}},
    {'$sort': SON([('count', -1), ('_id', -1)])}
]

In [16]:
import pprint

In [17]:
pprint.pprint(list(db.things.aggregate(pipeline)))

[{'_id': 'cat', 'count': 3},
 {'_id': 'dog', 'count': 2},
 {'_id': 'mouse', 'count': 1}]
